replacing fake implementation with variables

In [11]:
class Money: pass

class Sum:
    def __init__(self, augend: Money, addend: Money):
        self.augend = augend
        self.addend = addend

In [20]:
import abc

class Money: pass

class Expression(abc.ABC):
    def plus(self, addend: Money) -> Money:
        return Money(self.amount + addend.amount, self.currency)
    
    @abc.abstractmethod
    def reduce(self, to: str) -> Money: pass

class Sum(Expression):
    def __init__(self, augend: Money, addend: Money):
        self.augend = augend
        self.addend = addend
    
    def reduce(self, to: str):
        amount: int = self.augend.amount + self.addend.amount
        return Money(amount, to)

class Bank:
    def reduce(self, source: Expression, to: str) -> Money:
        # amount: int = source.augend.amount + source.addend.amount # move complex method into Sum class
        # if type(source) == Money: return source.reduce(to) # create reduce method in Expression and Money to remove type check
        return source.reduce(to)

class Money(Expression):
    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency

    @property
    def amount(self):
        return self._amount

    @property
    def currency(self):
        return self._currency

    @amount.setter
    def amount(self, amount: int):
        self._amount = amount

    @amount.deleter
    def amount(self):
        del self._amount

    def __eq__(self, o):
        return self._amount == o.amount \
            and self.currency == o.currency

    @staticmethod
    def dollar(amount: int):
        return Money(amount, "USD")

    @staticmethod
    def franc(amount: int):
        return Money(amount, "CHF")
    
    def times(self, multiplier: int):
        return Money(self.amount * multiplier, self.currency)
    
    @currency.getter
    def currency(self):
        return self._currency

    def plus(self, addend: Money) -> Expression:
        return Sum(self, addend)

    def reduce(self, to:str) -> Money:
        return self

In [21]:
import unittest

class TestMoneyExchange(unittest.TestCase):
    def test_multiplication(self):
        five: Money = Money.dollar(5)
        self.assertEqual(Money.dollar(10), five.times(2))
        self.assertEqual(Money.dollar(15), five.times(3))

    def test_currency(self):
        self.assertEqual("USD", Money.dollar(1).currency)
        self.assertEqual("CHF", Money.franc(1).currency)
    
    def test_equality(self):
        self.assertTrue(Money.dollar(5) == Money.dollar(5))
        self.assertFalse(Money.dollar(5) == Money.dollar(6))
        self.assertTrue(Money.franc(5) == Money.franc(5))
        self.assertFalse(Money.franc(5) == Money.franc(6))
        self.assertFalse(Money.franc(5) == Money.dollar(5))

    def test_different_class_equality(self):
        self.assertTrue(Money.franc(10) == Money.franc(10))
    
    def test_simple_addition(self):
        five: Money = Money.dollar(5)
        sum: Expression = five.plus(five)
        bank: Bank = Bank()
        reduced: Money = bank.reduce(sum, "USD")
        self.assertEqual(Money.dollar(10), reduced)
    
    def test_reduce_money(self):
        bank: Bank = Bank()
        result: Money = bank.reduce(Money.dollar(1), "USD")
        self.assertEqual(Money.dollar(1), result)

unittest.main(argv=[''], verbosity=2, exit=False)

test_currency (__main__.TestMoneyExchange) ... ok
test_different_class_equality (__main__.TestMoneyExchange) ... ok
test_equality (__main__.TestMoneyExchange) ... ok
test_multiplication (__main__.TestMoneyExchange) ... ok
test_reduce_money (__main__.TestMoneyExchange) ... ok
test_simple_addition (__main__.TestMoneyExchange) ... 

1 USD


ok

----------------------------------------------------------------------
Ran 6 tests in 0.004s

OK


The solution is to create an object that acts like a Money but represents the sum of two
Moneys.

One is to treat the sum
like a wallet: you can have several different notes of different denominations and currencies
in the same wallet.

Money is the atomic form of an expression. Operations result in Expressions.

Once the operation (such as adding up the value of a portfolio) is complete, the
resulting Expression can be reduced back to a single currency given a set of exchange rates.

```
def test_simple_addition():
    five: Money = Money.dollar()
    sum: Expression = five.plus(five)
    bank: Bank = Bank()
    reduced: Money = bank.reduce(sum, "USD")
    assertEqual(Money.dollar(10), reduced)
```

In [14]:
class Money: pass

In [15]:
import abc

class Expression(abc.ABC):
    def plus(self, addend: Money) -> Money:
        return Money(self.amount + addend.amount, self.currency)

class Bank:
    def reduce(self, source: Expression, to: str) -> Money:
        return Money.dollar(10)

class Money(Expression):
    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency

    @property
    def amount(self):
        return self._amount

    @property
    def currency(self):
        return self._currency

    @amount.setter
    def amount(self, amount: int):
        self._amount = amount

    @amount.deleter
    def amount(self):
        del self._amount

    def __eq__(self, o):
        return self._amount == o.amount \
            and self.currency == o.currency

    @staticmethod
    def dollar(amount: int):
        return Money(amount, "USD")

    @staticmethod
    def franc(amount: int):
        return Money(amount, "CHF")
    
    def times(self, multiplier: int):
        return Money(self.amount * multiplier, self.currency)
    
    @currency.getter
    def currency(self):
        return self._currency

    def plus(self, addend: Money) -> Money:
        return Money(self.amount + addend.amount, self.currency)

In [16]:
import unittest

class TestMoneyExchange(unittest.TestCase):
    def test_simple_addition(self):
        five: Money = Money.dollar(5)
        sum: Expression = five.plus(five)
        bank: Bank = Bank()
        reduced: Money = bank.reduce(sum, "USD")
        self.assertEqual(Money.dollar(10), reduced)

unittest.main(argv=[''], verbosity=2, exit=False)

test_simple_addition (__main__.TestMoneyExchange) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
